# MLflow example

To get started with mlflow, first we need to install mlflow: `pip install mlflow`

We then set up our notebook to track key parameters and metrics for each run. See link below for full example:

https://www.mlflow.org/docs/latest/tutorials-and-examples/index.html

## Notebook setup

In [28]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

In [3]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

## Helper functions

Define functions here

In [39]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

## Read in data

In [5]:
# Read the wine-quality csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    )

In [38]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

pickle.dump( train_x, open( "train_x.p", "wb" ) )
pickle.dump( train_y, open( "train_y.p", "wb" ) )
pickle.dump( test_x, open( "test_x.p", "wb" ) )
pickle.dump( test_y, open( "test_y.p", "wb" ) )

## Modelling with MLFlow

We will train models and evaluate them, and use MLFlow to keep a record of these models.
First, we will initialise a list of run names to be able identify these different runs later - we want to avoid naming multiple experiments with the same run name.

In [51]:
run_name_list = list()

The cell below has configurations for what parameters will be set / artifcats to be saved from this experiment. This will be a simple example, so few parameters and artifacts will be saved.

In [56]:
run_name = "mlflow-test4" # name of this experiment

# modelling parameters
alpha = 0.7
l1_ratio = 0.3

# run tags
tags = {
    "modelling_phase":"development"
    ,"model_type":"elasticnet"
}

# paths of artifacts to be saved
train_x_path = r"C:\Users\skyki\OneDrive\Data Science (Learning Materials)\Python_DataScience_Handbook\Mlflow Example\train_x.p"
train_y_path = r"C:\Users\skyki\OneDrive\Data Science (Learning Materials)\Python_DataScience_Handbook\Mlflow Example\train_y.p"
test_x_path = r"C:\Users\skyki\OneDrive\Data Science (Learning Materials)\Python_DataScience_Handbook\Mlflow Example\test_x.p"
test_y_path = r"C:\Users\skyki\OneDrive\Data Science (Learning Materials)\Python_DataScience_Handbook\Mlflow Example\test_y.p"

### Start experiment

Check if the run_name is a new run name. If it exists already, it will prompt to choose a different name.

In [57]:
if run_name in run_name_list:
    print("Set a new unique run name!")
else:
    run_name_list.append(run_name)
    mlflow.start_run(run_name=run_name)

Run code below to train model and run through the experiment.

In [60]:
mlflow.set_tags(tags)
mlflow.log_param("alpha", alpha)
mlflow.log_param("l1_ratio", l1_ratio)
mlflow.log_artifact(train_x_path)
mlflow.log_artifact(train_y_path)
mlflow.log_artifact(test_x_path)
mlflow.log_artifact(test_y_path)

lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
lr.fit(train_x, train_y)

predicted_qualities = lr.predict(test_x)

(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)


mlflow.log_metric("rmse", rmse)
mlflow.log_metric("r2", r2)
mlflow.log_metric("mae", mae)

tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

# Model registry does not work with file store
if tracking_url_type_store != "file":

    # Register the model
    # There are other ways to use the Model Registry, which depends on the use case,
    # please refer to the doc for more information:
    # https://mlflow.org/docs/latest/model-registry.html#api-workflow
    mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
else:
    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.700000, l1_ratio=0.300000):
  RMSE: 0.7573445868221795
  MAE: 0.6078932581456694
  R2: 0.1325223156013532


Finally, end the session.

In [59]:
mlflow.end_run()

## Viewing model runs in UI

To view the models that have been run, run `mlflow ui` in the terminal.